# NAVER TRENDS CRAWLING

* 공식 API 제공 (ID/SECRET 발급)
* 각 영화마다 키워드 설정하여 트랜드 조회 가능
* 기간별, 성별, 연령별, 채널별 조회 가능
* 동시 조회 가능 검색어 5개

[Reference](https://brunch.co.kr/@sukhyun9673/13)

#### IMPORT PACKAGE

In [1]:
import os
import sys
import urllib.request
import pandas as pd
import numpy as np
import json
from datetime import datetime

---

#### TEMPORARY CODE

In [ ]:
n_movie = len(movie_list)
for i in [*range(1, n_movie, 5)]:
    print(movie_list.iloc[i:i+5])
    print("==============================================")

In [ ]:
n_movie = len(movie_list)
for i in [*range(1, n_movie, 5)]:
    movie_subset = ["{\"groupName\": \"" + row["title"] + "\", \"keywords\": [\"" + row["title"] + "\"]}"  for idx, row in movie_list.iloc[i:i+5].iterrows()]

In [ ]:
print(j)

In [ ]:
client_id = "6GPEYsepjIil0OCSMnU5"
client_secret = "QUd5qt5QU9"
url = "https://openapi.naver.com/v1/datalab/search";

In [ ]:
response_body = get_response_body(body)
result = json.loads(response_body)
trends = get_trends(result)

In [ ]:
trends

In [ ]:
# OLD BODY
body =
"""
{\"startDate\":\"2018-12-01\", 
\"endDate\":\"2019-12-30\",
\"timeUnit\":\"date\",
\"keywordGroups\":
""" + "[" + ", ".join(movie_subset) + "]," + """
\"device\":\"pc\",
\"ages\":[\"1\",\"2\"],
\"gender\":\"f\"}
""";

---

#### DEFINE FUNCTIONS

In [2]:
def get_response_body(body):
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    request.add_header("Content-Type","application/json")
    response = urllib.request.urlopen(request, data=body.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        out = response.read().decode("utf-8")
        return(out)
    else:
        print("Error Code:" + rescode)

In [3]:
def get_trends(result):
    out = pd.DataFrame()
    for i in range(len(result["results"])):
        ttl = result["results"][i]["title"]
        data = result["results"][i]["data"]
        time = np.array([datetime.strptime(i["period"], "%Y-%m-%d") for i in data])
        value = np.array([j["ratio"] for j in data])
        title = np.repeat(ttl, len(value), axis=0)
        trend = pd.DataFrame({"Keyword": title, "Time": time, "Trend_index": value})
        out = pd.concat([out, trend])
    return(out.reset_index())

In [4]:
def recursive_min_max_search(init_data):
    def min_max_search(dat):
        n_movie = len(dat)
        rng = range(1, n_movie, 5)

        max_cand_out, min_cand_out = [] ,[]
        for i, idx in zip(rng, range(len(rng))):
            movie_subset = ["{\"groupName\": \"" + row["title"] + "\", \"keywords\": [\"" + row["title"] + "\"]}"  for idx, row in dat.iloc[i:i+5].iterrows()]
            body = """
            {\"startDate\":\"2018-12-01\", 
            \"endDate\":\"2019-12-30\",
            \"timeUnit\":\"date\",
            \"keywordGroups\":
            """ + "[" + ", ".join(movie_subset) + "]}"
            response_body = get_response_body(body)
            result = json.loads(response_body)
            trends_subset = get_trends(result)
            max, min = trends_subset.iloc[trends_subset['Trend_index'].idxmax()]["Keyword"], trends_subset.iloc[trends_subset['Trend_index'].idxmin()]["Keyword"]
            max_cand_out.append(max)
            min_cand_out.append(min)
            print(max, "|", min)
        return(max_cand_out, min_cand_out)

    x, y = min_max_search(init_data)
    if len(x) != 1 or len(y) != 1:
        print("================================================")
        return(recursive_min_max_search(init_data[init_data.title.isin(set([*x, *y]))]))
    else:
        return(x, y)

In [5]:
def get_trend_with_minmax(target_movie, min_movie, max_movie):
    movie_subset = ["{\"groupName\": \"" + movie_nm + "\", \"keywords\": [\"" + movie_nm + "\"]}" for movie_nm in set([target_movie, *min_movie, *max_movie])]
    body = """
    {\"startDate\":\"2018-12-01\", 
    \"endDate\":\"2019-12-30\",
    \"timeUnit\":\"date\",
    \"keywordGroups\":
    """ + "[" + ", ".join(movie_subset) + "]}"
    response_body = get_response_body(body)
    result = json.loads(response_body)
    trends_subset = get_trends(result)
    out = trends_subset[trends_subset.Keyword == target_movie]
    return(out)

---

#### EXEUCTION

In [6]:
client_id = "6GPEYsepjIil0OCSMnU5"
client_secret = "QUd5qt5QU9"
url = "https://openapi.naver.com/v1/datalab/search";

In [ ]:
client_id = "fe5kOiBVrMopPhGmaeeh"
client_secret = "zauzLV_BPn"
url = "https://openapi.naver.com/v1/datalab/search";

In [7]:
movie_list = pd.read_table("~/crawling/data/movie2019.dat",
                           sep = "|",
                           header=None,
                           names = ["title", "year", "month", "day"])

open_date = movie_list.year.str.replace("년", "") + "-" + movie_list.month.str.replace("월", "").str.zfill(2) + "-" +movie_list.day.str.replace("일", "").str.zfill(2)
movie_list["open_date"] = [datetime.strptime(i, "%Y-%m-%d") for i in open_date]
movie_list = movie_list.drop(['year', 'month', 'day'], axis=1)

In [8]:
min_movie, max_movie = recursive_min_max_search(movie_list)

HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
result_list = [get_trend_with_minmax(target_movie, min_movie, max_movie) for target_movie in [*movie_list.title]]

In [ ]:
result_full = pd.concat(result_list).drop("index", axis=1)

In [ ]:
result_merged = result_full.merge(movie_list, how="left", left_on="Keyword", right_on="title").drop("title", 1)

In [ ]:
result_merged["d_date"] = result_merged.Time - result_merged.open_date

In [ ]:
result_merged

In [ ]:
result_final.to_csv("~/crawling/data/result_final.csv")

---

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
mpl.rc('font', family='NanumGothic')
sns.set(rc={'figure.figsize':(15, 10)})

ax = sns.lineplot(x='d_date',
                  y='Trend_index',
                  hue='Keyword',
                  data=result_merged,
                  size=30, legend=False)

plt.title('Line Graph by seaborn', fontsize=20)
plt.ylabel('Trend', fontsize=14)
plt.xlabel('Date', fontsize=14)
plt.legend(fontsize=12, loc='best')
plt.show()

---

### 한글 폰트 문제 해결

In [ ]:
import matplotlib.font_manager as fm
font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
font_list

In [ ]:
fm.FontProperties(fname=font_list[10]).get_name()

In [ ]:
mpl.get_cachedir()

In [ ]:
# remove '/home/jupyter-tshahn2020/.cache/matplotlib/fontlist-v310.json'